In [3]:
#!python3
# plot vorticity in GOMb0.04 configuation
import sys
prfx='/home/abozec/PYTHON/'
sys.path.append(prfx)

#import myenv as my

from matplotlib.colors import BoundaryNorm
from scipy import ndimage
import netCDF4 as nc
import numpy as np

import math
import xarray as xr
import cartopy.crs as ccrs
import matplotlib as mpl
from matplotlib import pyplot as plt 
import matplotlib.pylab as plt
import matplotlib.path as mpath
from matplotlib.pyplot import figure
import cartopy.feature as cfeature
import pyproj
import utm
import pandas as pd 
import traceback 
from pathlib import Path  
from mpl_toolkits.basemap import Basemap
import metpy.calc as mpcalc
from metpy.units import units
import os  
import numpy.ma as ma
import glob
import datetime as dt
from netCDF4 import date2num,num2date

In [ ]:
ds_25 = xr.open_mfdataset('/Users/Anna/Desktop/MSMET/Thesis/WRFprepData/March2018HYCOM/020_archv.2018_084_*_3z.nc')#, combine = 'by_coords', concat_dim = 'time')

ds_25.to_netcdf('test_25.nc')
ds25 = xr.open_dataset('test_25.nc')


In [ ]:
#REAL RUN 
latbounds = [24, 29]
lonbounds = [-93, -87]

#TEST RUN
#latbounds = [25, 26]
#lonbounds = [-89, -88]

lats = ds25.Latitude.values[:]
lons = ds25.Longitude.values[:]

#set upper & lower bounds
latli = np.argmin(np.abs(lats-latbounds[0]))
latui = np.argmin(np.abs(lats-latbounds[1]))
lonli = np.argmin(np.abs(lons-lonbounds[0]))
lonui = np.argmin(np.abs(lons-lonbounds[1]))

lat_array=ds25.Latitude.values[latli:latui]
lon_array=ds25.Longitude.values[lonli:lonui]
u_subset_25=ds25.variables['u'][:,0,latli:latui, lonli:lonui]
meanU_25 = u_subset_25.mean('MT')
#meanU_25=meanU_25*0.54 - 0.19
meanU_25=meanU_25*0.75-0.15

#v current
v_subset_25=ds25.variables['v'][:,0,latli:latui, lonli:lonui]
meanV_25 = v_subset_25.mean('MT')
meanV_25=meanV_25*1.08-0.3
#meanV_25=meanV_25

#SST
SST_subset_25=ds25.variables['water_temp'][:,0,latli:latui, lonli:lonui]
meanSST_25 = SST_subset_25.mean('MT')


In [ ]:
#prfx='/p/home/abozec/'
#sys.path.append(prfx+'PYTHON/')
prfxs='/nexsan/people/abozec/'

# get grid
iot=prfxs+'/TSIS/GOMb0.04/topo/'
filet='regional.grid.a'
grid_data=iot+filet
#print(F"The fields available are: {my.hinfo.read_field_grid_names(grid_data)}")
fieldg=['plon','plat','pscx','pscy']
grid_field= my.hio.read_hycom_grid(grid_data, fieldg)
pscx=grid_field[fieldg[2]][:,:]
pscy=grid_field[fieldg[3]][:,:]
idm=pscx.shape[1] ; jdm=pscy.shape[0]
plon=grid_field[fieldg[0]][:,:]
plat=grid_field[fieldg[1]][:,:]

In [ ]:
year='2018'
#d=31
h=18
for d in [84]:
#for d in my.np.arange(21)+11:
   day='{:03d}'.format(d)
   hour='{:02d}'.format(h)

   ## NEMOSSH
   io_data = '/nexsan/archive/BOEM/GOMb0.04/data/'+year+'/'
   expt='031'
   file1=expt+'_archv.'+year+'_'+day+'_'+hour+'_3z.nc'
   file_data = io_data+file1
   print(file_data)

   # get metadata of .[ab]
   hycom_field= nc.Dataset(file_data)

   # get u and v
   ubc=hycom_field['u'][0,0,0:jdm,0:idm]
   vbc=hycom_field['v'][0,0,0:jdm,0:idm]
   print(ubc.shape)
   u=ubc.data
   v=vbc.data
   u[u > 1e10]=np.nan
   v[v > 1e10]=np.nan

   # get vorticity
   vort_ssh=my.np.zeros([jdm,idm])
   vort_ssh[:,:]=(v - my.np.roll(v,1,axis=1))/pscx - (u - my.np.roll(u,1,axis=0))/pscy
   vort_ssh[abs(vort_ssh) > 1e10 ] = 0.

In [ ]:
 # colorbar
   cmap_u=my.mygc.get_cmapbluered()
   N=255
   levels_u=my.np.linspace(-50,50,101)
   #levels_u=my.np.linspace(-100,100,101)
   #print(levels_u)
   norm = BoundaryNorm(levels_u, ncolors=cmap_u.N, clip=True)

   vort_ssh=my.np.nan_to_num(vort_ssh,nan=0.)
   y = ndimage.uniform_filter(vort_ssh*1e6,size=4)


In [ ]:
# plot the field
   # change land color
   my.plot.rc.landcolor=[150/256,150/256,150/256]
   my.plot.rc.reso = 'med'  # use higher res for zoomed in geographic features
#   proj = my.plot.Proj('mill', lonlim=(-99, -78), latlim=(18, 32), basemap=True)
   fig, axs = my.plot.subplots(nrows=1,ncols=1,proj='cyl',refwidth='12cm',bottom='4em')
   #CS2=axs[0].contourf(plon,plat,vort*1e6, cmap=cmap_u, extend='both',\
   #              levels=levels_u)
   CS2 = axs[0].pcolormesh(plon[0,:], plat[:,0], y, cmap=cmap_u, norm=norm, \
                               shading='auto', extend='both')

   q = axs[0].quiver(plon[0,::4],plat[::4,0], u[::4,::4],v[::4,::4],scale=1.,scale_units='x',color='gray7')
   axs[0].quiverkey(q, X=0.2, Y=-0.075, U=1., label='1. m/s', labelpos='E')

   axs[0].format(latlines=1, lonlines=1,lonlim=(-93, -87.5), latlim=(24.5,29),\
                   labels=True,land=True,\
              title='GOMb0.04-TSIS Vorticity (1e6 s-1) Year:'+year+' Day:'+day+' Hour:'+hour)

   #axs[0].contour(lon1,lat1,vort*1e5, color='k', levels=levels_u[::2],lw=0.2)
   axs[0].colorbar(CS2,loc='r',ticks=10)

   ps_dir='/home/abozec/PYTHON/PNG/'
   file_png='Test_vort-vec_gomb4-tsis_'+year+'_'+day+'_'+hour+'.png'
   print(ps_dir+file_png)
   fig.savefig(ps_dir+file_png,dpi=150,\
               facecolor='w', edgecolor='w',transparent=False,bbox_inches='tight') ## .pdf,.eps
   #my.plot.close() ##initcond
   my.plot.show()

